This tutorial can be downloaded [link](https://github.com/west-code-development/West/raw/master/Doc/tutorials/advanced/advanced_001.ipynb).

# Advanced Tutorial 1: Symmetry Analysis in QDET Calculations

We explain how to identify the symmetry of excited many-body states calculated with the Quantum Defect Embedding Theory (QDET) using the point symmetry of the defect.

In this tutorial, we focus on the $\mathrm{NV^-}$ center in diamond.

## Step 1: Mean-field starting point

Similarly to the case of a standard QDET calculation, the workflow starts with the calculation of the mean-field starting point. In order to perform the symmetry analysis, we additionally write the single-particle wavefunctions to cube files.

Download the following files in your working directory:

In [1]:
%%bash
wget -N -q https://west-code.org/doc/training/nv_diamond_63/pw.in
wget -N -q http://www.quantum-simulation.org/potentials/sg15_oncv/upf/C_ONCV_PBE-1.2.upf
wget -N -q http://www.quantum-simulation.org/potentials/sg15_oncv/upf/N_ONCV_PBE-1.2.upf

We can now inspect the `pw.in` file, the input for the `pw.x` code:

In [2]:
%%bash
cat pw.in

&CONTROL
calculation = 'scf'
pseudo_dir = './'
/
&SYSTEM
input_dft = 'PBE'
ibrav = 0
ecutwfc = 50
tot_charge = -1
nspin = 1
nbnd = 176
occupations = 'from_input'
nat = 63
ntyp = 2
/
&ELECTRONS
diago_full_acc = .true.
/
K_POINTS gamma
CELL_PARAMETERS angstrom
7.136012  0.000000  0.000000
0.000000  7.136012  0.000000
0.000000  0.000000  7.136012
ATOMIC_SPECIES
C  12.01099968  C_ONCV_PBE-1.2.upf
N  14.00699997  N_ONCV_PBE-1.2.upf
ATOMIC_POSITIONS crystal
C    0.99996000  0.99996000  0.99996000
C    0.12495000  0.12495000  0.12495000
C    0.99905000  0.25039000  0.25039000
C    0.12350000  0.37499000  0.37499000
C    0.25039000  0.99905000  0.25039000
C    0.37499000  0.12350000  0.37499000
C    0.25039000  0.25039000  0.99905000
C    0.37499000  0.37499000  0.12350000
C    0.00146000  0.00146000  0.50100000
C    0.12510000  0.12510000  0.62503000
C    0.00102000  0.24944000  0.74960000
C    0.12614000  0.37542000  0.87402000
C    0.24944000  0.00102000  0.74960000
C    0.37542000  0.12614

We run `pw.x` on 2 cores.

In [ ]:
%%bash
mpirun -n 2 pw.x -i pw.in > pw.out

We use `westpp.x` to write the wavefunctions to cube files. Create the input file `westpp.in`:

In [3]:
import yaml

d = {}
d["westpp_control"] = {}
d["westpp_control"]["westpp_calculation"] = "W"
d["westpp_control"]["westpp_range"] = [1, 176]
d["westpp_control"]["westpp_format"] = "C"
d["westpp_control"]["westpp_sign"] = True
        
with open("westpp.in", "w") as f:
    yaml.dump(d, f, sort_keys=False)

In [4]:
%%bash
cat westpp.in

westpp_control:
  westpp_calculation: W
  westpp_range:
  - 1
  - 176
  westpp_format: C
  westpp_sign: true


We run `westpp.x` on 2 cores.

In [ ]:
%%bash
mpirun -n 2 westpp.x -i westpp.in > westpp.out

## Step 2: Definition of the point group

We now define the point group. A point group is characterized by two quantities:

- a set of symmetry operations (identity, reflection, and rotation operations)
- a corresponding character table

In [5]:
import numpy as np
from westpy.qdet.symm import PointGroup, PointGroupOperation, PointGroupRotation, PointGroupReflection

sq3 = np.sqrt(3)

# the origin is set to the nitrogen site.
# The prefactor corresponds to the real-space FFT grid along each direction.
origin = 64 * np.array([0.48731,  0.48731,  0.48731])

point_group = PointGroup(
    name="C3v",
    operations={
        "E": PointGroupOperation(T=np.eye(4)),
        "C3_1": PointGroupRotation(rotvec=2 * np.pi / 3 * np.array([1/sq3, 1/sq3, 1/sq3]),
                                   origin=origin),
        "C3_2": PointGroupRotation(rotvec=4 * np.pi / 3 * np.array([1/sq3, 1/sq3, 1/sq3]),
                                   origin=origin),
        "Cv_1": PointGroupReflection(normal=(1, -1, 0), origin=origin),
        "Cv_2": PointGroupReflection(normal=(0, -1, 1), origin=origin),
        "Cv_3": PointGroupReflection(normal=(-1, 0, 1), origin=origin)
    },
    ctable={
        "A1": [1, 1, 1, 1, 1, 1],
        "A2": [1, 1, 1, -1, -1, -1],
        "E": [2, -1, -1, 0, 0, 0]})

 
 _    _ _____ _____ _____            
| |  | |  ___/  ___|_   _|           
| |  | | |__ \ `--.  | |_ __  _   _  
| |/\| |  __| `--. \ | | '_ \| | | | 
\  /\  / |___/\__/ / | | |_) | |_| | 
 \/  \/\____/\____/  \_/ .__/ \__, | 
                       | |     __/ | 
                       |_|    |___/  
 
WEST version     :  6.0.0
Today            :  2024-06-19 10:30:15.939195


## Step 3: Diagonalization of QDET Hamiltonian

In this tutorial, we focus on the symmetry analysis with QDET. To avoid costly calculations of the matrix elements of the effective Hamiltonian, we simply download the output file of the QDET calculation. To learn how to perform QDET calculations, see [Basic Tutorial 5](https://github.com/west-code-development/West/raw/master/Doc/tutorials/basic/basic_005.ipynb).

In [6]:
%%bash
mkdir -p west.wfreq.save
wget -N -q https://west-code.org/doc/training/nv_diamond_63/wfreq.json -O west.wfreq.save/wfreq.json

To add point group analysis to QDET, we pass the point group object defined above to the Hamiltonian object. Additionally, we provide the list of filenames for the wavefunctions.

The code will apply the symmetry operations to the Kohn-Sham wavefunctions stored in the cube files and determine the character of these wavefunctions. After the diagonalization of the effective Hamiltonian, the character of the many-body states can then be inferred from the character of the wavefunctions.

In [ ]:
%%bash
wget -N -q https://west-code.org/doc/training/nv_diamond_63/wfcK000001B000087.cube
wget -N -q https://west-code.org/doc/training/nv_diamond_63/wfcK000001B000122.cube
wget -N -q https://west-code.org/doc/training/nv_diamond_63/wfcK000001B000123.cube
wget -N -q https://west-code.org/doc/training/nv_diamond_63/wfcK000001B000126.cube
wget -N -q https://west-code.org/doc/training/nv_diamond_63/wfcK000001B000127.cube
wget -N -q https://west-code.org/doc/training/nv_diamond_63/wfcK000001B000128.cube

In [7]:
# the diagonalization is the same as without the symmetry analysis
from westpy.qdet import QDETResult

# list of cube files for the wavefunctions
wfct_filenames = ["wfcK000001B000087.cube", "wfcK000001B000122.cube", "wfcK000001B000123.cube",
                  "wfcK000001B000126.cube", "wfcK000001B000127.cube", "wfcK000001B000128.cube"]

# construct object for effective Hamiltonian
effective_hamiltonian = QDETResult(filename="west.wfreq.save/wfreq.json", point_group=point_group, wfct_filenames=wfct_filenames)

# diagonalize Hamiltonian
solution = effective_hamiltonian.solve()

PointGroupRep: rep matrices are orthogonal
Irrep of orbitals: ['A1(1.00)', 'A1(1.00)', 'A1(1.00)', 'A1(1.00)', 'E(1.00)', 'E(1.00)']
Solutions are projected onto irreps of C3v group
-----------------------------------------------------
Building effective Hamiltonian...
nspin: 1
occupations: [[2. 2. 2. 2. 1. 1.]]


diag[1RDM - 1RDM(GS)]                                 
  E [eV] char                    87    122    123    126   127   128
0  0.000  3A2                 0.000  0.000  0.000  0.000 0.000 0.000
1  0.436   1E                -0.001 -0.009 -0.018 -0.067 0.004 0.091
2  0.436   1E                -0.001 -0.009 -0.018 -0.067 0.092 0.002
3  1.251  1A1                -0.002 -0.019 -0.023 -0.067 0.054 0.057
4  1.939   3E                -0.003 -0.010 -0.127 -0.860 1.000 0.000
5  1.940   3E                -0.003 -0.010 -0.127 -0.860 0.000 1.000
6  2.935   1E                -0.000 -0.032 -0.043 -0.855 0.929 0.002
7  2.936   1E                -0.000 -0.032 -0.043 -0.855 0.002 0.929
8  4.661  1A1                -0.006 -0.054 -0.188 -1.672 0.960 0.960
9  5.080   3E                -0.014 -0.698 -0.213 -0.075 1.000 0.000

-----------------------------------------------------
